## **실행 전에**

reversed_data_after_preprocessing.csv 추가

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 60 (delta 23), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [4]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [5]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.7MB/s 
     |████████████████████████████████| 3.8MB 36.6MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-08-30 12:35:05--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.0, 18.205.93.1, 18.205.93.2, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.0|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?S

# 전처리

In [6]:
import pandas as pd
import csv
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Mecab

In [10]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/5.자연어처리&추천시스템(김준태)/mini_pro/reversed_data_after_preprocessing.csv')
data

,title,content
0,서해상으로 북상 중인 태풍 바비 피해 줄이려면,Q 특히 농어촌에서 어떤 피해가 발생할지가 걱정인데요 Q 도심 집에 계신 ...
1,새벽 4시 5시 사이 서울 근접 이 시각 서울시 재난안전상황실,만약 이런 피해가 발생하면 가까운 구청 상황실이나 다산콜센터 등에 연락하시면 됩니...
2,코로나 때문에 봉사시간 못 채워 졸업 막힌 대학생들,정부 차원의 대책 마련이 필요하다는 의견도 나온다 코로나19라는 비상 상황인 만...
3,성인 남자가 서 있기 힘들 정도 태풍 바비 군산 도착,지금까지 전북 군산 비응항에서 YTN 김민성 입니다 이번 태풍 아무쪼록 큰 피해 ...
4,위험 반원 인천 초비상 초속 25m 넘으면 영종대교 통제,또 초속 25m 이상 바람이 불 경우 인천공항으로 향하는 영종대교와 인천대교 차량 ...
...,...,...
9370,깜깜이 감염 확진자 확산하는 원주시 다중이용시설 폐쇄,시는 전문 역학조사팀이 CCTV 확인과 카드 사용 조사 등 정밀한 역학조사를...
9371,원주 코로나19 확진 잇달아 5명 추가 확진,도교육청은 오늘 원주교육지원청에서 원주권 유치원 초 중 고 특수학교 전체 비상교...
9372,경기 어제 신규확진 109명 코로나사태 이후 일일 최다,신규 확진자 중 감염 경로가 확인되지 않아 조사 중 으로 분류된 환자가 2...
9373,방역당국 사랑제일교회 밤샘대치 21일 명단확보 재시도 위해 대기 중,현재 방역당국 관계자들은 오전 9시 현재까지 현장에 남아 역학조사 재개를 위한 대기...


토큰화 + 불용어제거

In [11]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다',
           '인데요','다','을','습니다','Q','이다','보인다','한다','했다','였다','이다']

In [53]:
encoder_input, decoder_input, decoder_output = [], [], []
word_tag = ['NNG', 'NNP', 'NNB', 'NNB', 'NNBC', 'NR', 'VV', 'VA']

content_tokenizer = Mecab()
title_tokenizer = Mecab()

for line in data['content']:
    temp = content_tokenizer.pos(str(line))
    temp = [word for word in temp if word[1] in word_tag]
    temp = [word[0] for word in temp if not word[0] in stopwords] # 불용어 제거
    encoder_input.append(temp)

for line in data['title']:
    temp = []
    temp.append('<start>')
    for word in title_tokenizer.pos(str(line)):
      if word[1] in word_tag:
          if word not in stopwords: # 불용어 제거
            temp.append(word[0]) 
    decoder_input.append(temp)
  
for line in data['title']:
    temp = []
    for word in title_tokenizer.pos(str(line)):
      if word[1] in word_tag:
        if word[0] not in stopwords: # 불용어 제거
          temp.append(word[0]) 
    temp.append('<end>')
    decoder_output.append(temp)

In [54]:
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

[['농어촌', '피해', '발생', '걱정', '도심', '집', '분', '대비', '고층', '아파트', '유리창', '깨지', '피해', '태풍', '때', '발생', '그때', '신문지', '테이프', '예방', '되', '요즘', '창틀', '창틀', '사이', '부분', '고정', '중요', '하', '태풍', '일', '때', '운전', '하', '좋', '수', '운전', '되', '경우', '있', '때', '조심', '도심', '강풍', '피해', '대표', '간판', '교회', '첨탑', '무너지', '경우', '있', '서해', '태풍', '올라오', '피해', '발생', '이번', '바람', '피해', '것', '같', '강원', '대학교', '방재', '전문', '대학원', '김병식', '교수', '모시', '태풍', '피해', '대응', '하', '얘기'], ['피해', '발생', '구청', '상황실', '다산', '콜', '센터', '등', '연락', '정전', '피', '수', '있', '공사장', '가림막', '현수막', '건물', '간판', '아파트', '유리창', '등', '바람', '날아가', '사람', '수', '있', '외출', '하', '좋', '하', '경우', '건물', '공사장', '주변', '피하', '좋', '강풍', '피해', '있', '수', '있', '대비', '하', '서울시', '신호등', '가로등', '같', '도로', '교통', '시설물', '가로수', '강풍', '수', '있', '경고', '태풍', '지나가', '시민', '조심', '서울', '시내', '태풍', '취약', '지점', '레이더', '영상', '보', '태풍', '지나가', '오늘', '새벽', '서울', '비', '예보', '서울시', '공공', '자전거', '릉', '운행', '저녁', '시', '태풍', '소멸', '때', '중단', '서울시', '개', '자치구', '있', '코로나', '실외', '선별', '진료소',

정수인코딩

In [55]:
import nltk
from nltk import FreqDist
import numpy as np

encoder_vocab = FreqDist(np.hstack(encoder_input))
decoder_vocab = FreqDist(np.append(np.hstack(decoder_input), ['<end>'] * len(decoder_output)))
# print(encoder_input[0])
print(len(encoder_vocab))
print(len(decoder_vocab))

29511
7563


In [56]:
# 빈도수 적게 나온 단어 빼기
encoder_vocab = [(morph,freq) for morph, freq in encoder_vocab.most_common() if freq > 3]
decoder_vocab = [(morph,freq) for morph, freq in decoder_vocab.most_common() if freq > 3]

print(len(encoder_vocab))
print(len(decoder_vocab))

15135
2795


In [57]:
encoder_word_to_index = {word[0]: index + 2 for index, word in enumerate(encoder_vocab)}
decoder_word_to_index = {word[0]: index + 2 for index, word in enumerate(decoder_vocab)}

In [58]:
len(decoder_word_to_index)

2795

In [59]:
# 특수 인덱스
encoder_word_to_index['unk'] = 1

encoder_encoded_input = []
for line in encoder_input:
    tmp = []
    for word in line:
        try:
            # 각 글자를 해당하는 정수로 변환한다.
            tmp.append(encoder_word_to_index[word])
        except KeyError: 
            # 단어 집합에 없는 단어일 경우 unk로 대체된다.
            tmp.append(encoder_word_to_index['unk'])

    encoder_encoded_input.append(tmp)

# 기존의 리뷰가 성공적으로 encoding 되었는지 확인해보기
print(encoder_input[1])
print(encoder_encoded_input[1])

['피해', '발생', '구청', '상황실', '다산', '콜', '센터', '등', '연락', '정전', '피', '수', '있', '공사장', '가림막', '현수막', '건물', '간판', '아파트', '유리창', '등', '바람', '날아가', '사람', '수', '있', '외출', '하', '좋', '하', '경우', '건물', '공사장', '주변', '피하', '좋', '강풍', '피해', '있', '수', '있', '대비', '하', '서울시', '신호등', '가로등', '같', '도로', '교통', '시설물', '가로수', '강풍', '수', '있', '경고', '태풍', '지나가', '시민', '조심', '서울', '시내', '태풍', '취약', '지점', '레이더', '영상', '보', '태풍', '지나가', '오늘', '새벽', '서울', '비', '예보', '서울시', '공공', '자전거', '릉', '운행', '저녁', '시', '태풍', '소멸', '때', '중단', '서울시', '개', '자치구', '있', '코로나', '실외', '선별', '진료소', '오전', '시', '오후', '시', '한시', '중단', '집계', '피해', '없', '새벽', '시', '시', '사이', '태풍', '서울', '만큼', '상황', '예의', '주시', '서울', '평균', '풍속', '초속', '정도', '서울시', '저녁', '시', '비상', '대응', '단계', '발령', '태풍', '피해', '대비', '서울시', '내', '곳곳', '카메라', '실시간', '확인', '서울', '비', '오', '서울시', '태풍', '대비', '분주', '것', '같', '상황', '어떻', '곳', '서울', '시청', '마련', '재난', '안전', '상황실', '이번', '태풍', '대비', '서울시', '재난', '안전', '상황실']
[78, 25, 822, 1526, 12474, 1961, 184, 6, 641, 2257, 1641, 1

In [60]:
# 단어들에 순차적으로 2 ~ 까지의 인덱스를 부여한다(unknown=1)
decoder_word_to_index['unk'] = 1

decoder_encoded_input = []
for line in decoder_input:
    tmp = []
    for word in line:
        try:
            tmp.append(decoder_word_to_index[word])
        except KeyError: 
            tmp.append(decoder_word_to_index['unk'])

    decoder_encoded_input.append(tmp)

print(decoder_input[1])
print(decoder_encoded_input[1])

['<start>', '새벽', '시', '시', '사이', '서울', '근접', '시각', '서울시', '재난', '안전', '상황실']
[2, 450, 53, 53, 597, 16, 252, 420, 85, 103, 390, 1]


In [61]:
decoder_encoded_output = []
for line in decoder_output:
    tmp = []
    for word in line:
        try:
            tmp.append(decoder_word_to_index[word])
        except KeyError:
            tmp.append(decoder_word_to_index['unk'])

    decoder_encoded_output.append(tmp)

print(decoder_output[1])
print(decoder_encoded_output[1])

['새벽', '시', '시', '사이', '서울', '근접', '시각', '서울시', '재난', '안전', '상황실', '<end>']
[450, 53, 53, 597, 16, 252, 420, 85, 103, 390, 1, 3]


In [62]:
# 기존의 리뷰가 성공적으로 encoding 되었는지 확인해보기
print(encoder_encoded_input[0])
print(decoder_encoded_input[0])
print(decoder_encoded_output[0])

[4703, 78, 25, 525, 1176, 197, 65, 302, 6599, 486, 2741, 4234, 78, 31, 68, 25, 1407, 5778, 3483, 524, 77, 2532, 4172, 4172, 308, 322, 1662, 392, 11, 31, 2, 68, 1802, 11, 351, 10, 1802, 77, 57, 7, 68, 1702, 1176, 306, 78, 206, 1499, 13, 5470, 2395, 57, 7, 717, 31, 1792, 78, 25, 86, 193, 78, 5, 59, 496, 1004, 4891, 655, 1551, 1, 195, 4704, 31, 78, 176, 11, 637]
[2, 685, 91, 17, 10, 27, 61, 1613]
[685, 91, 17, 10, 27, 61, 1613, 3]


패딩

In [63]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# maxlen 없어도 알아서 잘 패딩합니다

encoder_input = pad_sequences(encoder_encoded_input, padding="post")
decoder_input = pad_sequences(decoder_encoded_input, padding="post")
decoder_output = pad_sequences(decoder_encoded_output, padding="post")

In [64]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_output.shape)
print(encoder_input[:3])
print(decoder_input[:3])
print(decoder_output[:3])

(9375, 2640)
(9375, 19)
(9375, 19)
[[4703   78   25 ...    0    0    0]
 [  78   25  822 ...    0    0    0]
 [  24  905  104 ...    0    0    0]]
[[   2  685   91   17   10   27   61 1613    0    0    0    0    0    0
     0    0    0    0    0]
 [   2  450   53   53  597   16  252  420   85  103  390    1    0    0
     0    0    0    0    0]
 [   2    6  352 2046  166 2369 2047    0    0    0    0    0    0    0
     0    0    0    0    0]]
[[ 685   91   17   10   27   61 1613    3    0    0    0    0    0    0
     0    0    0    0    0]
 [ 450   53   53  597   16  252  420   85  103  390    1    3    0    0
     0    0    0    0    0]
 [   6  352 2046  166 2369 2047    3    0    0    0    0    0    0    0
     0    0    0    0    0]]


테스트-트레인셋 분리

In [65]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[6915 3860 1037 ... 2780 1149 8852]


In [66]:
# 3:1로 나눴습니다
test_size = int(encoder_input.shape[0] * 0.25)
train_size = encoder_input.shape[0] - test_size
train_idx = indices[:train_size]
test_idx = indices[train_size:]
encoder_input_train = encoder_input[train_idx]
decoder_input_train = decoder_input[train_idx]
decoder_output_train = decoder_output[train_idx]

encoder_input_test = encoder_input[test_idx]
decoder_input_test = decoder_input[test_idx]
decoder_output_test = decoder_output[test_idx]

In [67]:
print(encoder_input_train.shape)
print(encoder_input_test.shape)

(7032, 2640)
(2343, 2640)


# 학습

In [68]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

인코더

In [69]:
encoder_inputs = Input(shape=(encoder_input_train.shape[1],))
encoder_embed = Embedding(len(encoder_word_to_index)+1, 50)(encoder_inputs)
encoder_mask = Masking(mask_value=0)(encoder_embed)
encoder_outputs, h_state, c_state = LSTM(50, return_state=True)(encoder_mask)

디코더

In [70]:
decoder_inputs = Input(shape=(decoder_input_train.shape[1],))
decoder_embed = Embedding(len(decoder_word_to_index)+1, 50)(decoder_inputs)
decoder_mask = Masking(mask_value=0)(decoder_embed)

decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_mask, initial_state=[h_state, c_state])
decoder_dense = Dense(len(decoder_word_to_index)+1, activation='softmax')
decoder_softmax_outputs = decoder_dense(decoder_outputs)

In [71]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# validation loss를 계속 보다가 5회 이상 loss가 증가하면, 과적합될 수 있으므로 학습을 조기 종료하겠다.
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=8)
# epoch를 반복하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트로 저장
# 정확도가 낮아지면 모델 버려라
model_check = ModelCheckpoint('mini_project_model_01.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [72]:
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_output_train, 
          validation_data = ([encoder_input_test, decoder_input_test], decoder_output_test), 
          batch_size = 128, epochs = 50, callbacks=[early_stop, model_check])

Epoch 1/50
55/55 [==============================] - ETA: 0s - loss: 5.6365 - acc: 0.5041
Epoch 00001: val_acc improved from -inf to 0.51178, saving model to mini_project_model_01.h5
55/55 [==============================] - 243s 4s/step - loss: 5.6365 - acc: 0.5041 - val_loss: 4.0674 - val_acc: 0.5118
Epoch 2/50
55/55 [==============================] - ETA: 0s - loss: 3.5810 - acc: 0.5108
Epoch 00002: val_acc did not improve from 0.51178
55/55 [==============================] - 243s 4s/step - loss: 3.5810 - acc: 0.5108 - val_loss: 3.3127 - val_acc: 0.5118
Epoch 3/50
55/55 [==============================] - ETA: 0s - loss: 3.1849 - acc: 0.5248
Epoch 00003: val_acc improved from 0.51178 to 0.54539, saving model to mini_project_model_01.h5
55/55 [==============================] - 243s 4s/step - loss: 3.1849 - acc: 0.5248 - val_loss: 3.0861 - val_acc: 0.5454
Epoch 4/50
55/55 [==============================] - ETA: 0s - loss: 3.0218 - acc: 0.5695
Epoch 00004: val_acc improved from 0.54539 to

KeyboardInterrupt: ignored

# 예측

In [79]:
index_to_title = {index: title for title, index in decoder_word_to_index.items()}

In [80]:
encoder_model = Model(encoder_inputs, [h_state, c_state])

encoder_h_state = Input(shape=(50,))
encoder_c_state = Input(shape=(50,))
pd_decoder_outputs, pd_h_state, pd_c_state = decoder_lstm(decoder_mask, initial_state=[encoder_h_state, encoder_c_state])
pd_decoder_softmax_outputs = decoder_dense(pd_decoder_outputs)

decoder_model = Model([decoder_inputs] + [encoder_h_state, encoder_c_state], [pd_decoder_softmax_outputs] + [pd_h_state, pd_c_state])

In [86]:
def decode_sequence(input_seq):
    state_value = encoder_model.predict(input_seq)

    predicted_seq = np.zeros((1, 1))
    predicted_seq[0, 0] = index_to_title['<start>']

    decode_stc = []

    while True:
        output_words, h, c = decoder_model.predict([predicted_seq] + state_value)

        predicted_word = index_to_title[np.argmax(output_words[0,0])]

        if predicted_word == '<end>':
            break

        decode_stc.append(predicted_word)

        predicted_seq = np.zeros((1,1))
        predicted_seq[0, 0] = np.argmax(output_words[0, 0])

        states_valus = [h, c]

    return decoded_sentence

In [87]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2text(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + content_index_to_word[i]+' '
    return temp

# 요약문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2summary(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=title_word_to_index['<start>']) and i!=title_word_to_index['<end>']):
            temp = temp + title_index_to_word[i] + ' '
    return temp

In [88]:
for i in encoder_input_test:
    print("원문 : ",seq2text(i))
    print("실제 요약문 :",seq2summary(decoder_input_test[i]))
    print("예측 요약문 :",decode_sequence(decoder_output_test[i].reshape(1, text_max_len)))
    print("\n")

KeyError: ignored

In [81]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    # e_out, e_h, e_c = encoder_model.predict(input_seq)
    state_value = encoder_model.predict(input_seq)

     # <SOS>에 해당하는 토큰 생성
    target_seq = np.zeros((1,1,len(encoder_vocab)))
    target_seq[0, 0, decoder_encoded_output] = title_word_to_index['<start>']

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition: # stop_condition이 True가 될 때까지 루프 반복
        output_tokens, h, c = decoder_model.predict([target_seq] + state_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target
        sampled_token = title_index_to_word[sampled_token_index]

        if (sampled_token!='<end>'):
            decoded_sentence += ' '+sampled_token

        #  <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_token == '<end>'  or len(decoded_sentence.split()) >= (summary_max_len-1)):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 상태를 업데이트 합니다.
        e_h, e_c = h, c

    return decoded_sentence